# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [67]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import TimeSeriesSplit


## Step 2: Load Chess Data

Load cleaned data from csv file


In [68]:
df = pd.read_csv("/Users/riteshbhandari/Documents/Dokumentit – Ritesh - MacBook Pro/GitHub/Chess-engine/src/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2.1:  Further Pre-processing


In [69]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [70]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[1730, 348, 1000, 990, 665, 812, 1101, 904, 1335, 1174]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [71]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [72]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [73]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 1730, 1), (0, 348, 0), (1, 1000, 1), (0, 990, 0), (1, 665, 1), (0, 812, 0), (1, 1101, 1), (0, 904, 0), (1, 1335, 1), (0, 1174, 0), (1, 1664, 1), (0, 673, 0), (1, 723, 1), (0, 1502, 0), (1, 1387, 1), (0, 363, 0), (1, 1494, 1), (0, 1679, 0), (1, 306, 1), (0, 1367, 0), (1, 1075, 1), (0, 1563, 0), (1, 1066, 1), (0, 650, 0), (1, 592, 1), (0, 525, 0), (1, 1635, 1), (0, 883, 0), (1, 205, 1), (0, 696, 0), (1, 1075, 1), (0, 1340, 0), (1, 19, 1), (0, 1847, 0), (1, 488, 1), (0, 242, 0), (1, 1848, 1), (0, 396, 0), (1, 585, 1), (0, 1435, 0), (1, 475, 1), (0, 1082, 0), (1, 11, 1), (0, 729, 0), (1, 488, 1), (0, 394, 0), (1, 337, 1), (0, 926, 0), (1, 821, 1), (0, 1070, 0), (1, 1024, 1), (0, 1155, 0), (1, 880, 1), (0, 1679, 0), (1, 1723, 1), (0, 1068, 0), (1, 488, 1), (0, 1581, 0), (1, 585, 1), (0, 1370, 0), (1, 968, 1), (0, 192, 0), (1, 14, 1), (0, 1061, 0), (1, 690, 1), (0, 1463, 0), (1, 1584, 1), (0, 1153, 0), (1, 968, 1), (0, 1463, 0), (1, 1584, 1), (0, 1153, 0), (1, 1703, 1), (0, 1775, 0), (1

In [74]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 1730, 1), (0, 348, 0), (1, 1000, 1), (0, ..."
1,123118510404,"[(1, 1000, 0), (0, 1764, 1), (1, 1812, 0), (0,..."
2,123118790014,"[(1, 1730, 0), (0, 1335, 1), (1, 1101, 0), (0,..."
3,123158939328,"[(1, 1730, 0), (0, 1812, 1), (1, 1335, 0), (0,..."
4,123160166430,"[(1, 1730, 1), (0, 1335, 0), (1, 1442, 1), (0,..."


## Step 2.2: Data Pipeline

### Dataset Class
Organize chess games into training sequences with sliding windows and padding.

In [75]:
unique_moves = set()
for game in df["game_data"]:
    for (color, move, your_move) in game:
        unique_moves.add(move)

print(f"Number of unique moves: {len(unique_moves)}")
print(f"Move range: {min(unique_moves)} to {max(unique_moves)}")

Number of unique moves: 1927
Move range: 0 to 1926


In [76]:
VOCAB_SIZE = 1928 # 1927 moves and one padding token
MAX_SEQUENCE_LENGTH = 6 # for now 
PAD_TOKEN = 1927 # for not known moves

class chessdataset(Dataset):
    
    def __init__(self, games_data, max_seq_len = 6):
        self.max_seq_len = max_seq_len
        self.pad_token = PAD_TOKEN
        self.sequences = []
        self.targets = []
        
        # loop through each games 
        for game in games_data:
            for i in  range(len(game) -1):
                start_idx = max(0,i-5)
                sequence = game[start_idx : i+1]  
            
                while len(sequence) <6:
                    sequence.insert(0,(0,1927,0))
                   
                target = game[i+1][1] # the real seventh move 
                self.sequences.append(sequence)
                self.targets.append(target)



    # training samples
    def __len__ (self):
        return len(self.targets)

        
      # creating batches           
    def __getitem__ (self, idx):
        sequence = self.sequences[idx]
        target = self.targets[idx]

        # saving training samples seperately to it's own category
        colors = [move_tuple[0] for move_tuple in sequence]
        moves = [move_tuple[1] for move_tuple in sequence]
        theory = [move_tuple[2] for move_tuple in sequence]
        
         # total 60 600 training samples
        return {
            'colors': torch.tensor(colors),
            'moves': torch.tensor(moves),
            'theory': torch.tensor(theory),
            'target': torch.tensor(target), 
    }                

In [77]:
# Create the dataset
dataset = chessdataset(df['game_data'])

# Check how many training examples
print(f"Total training examples: {len(dataset)}")

# Get one example
example = dataset[0]
print(f"\nFirst training example:")
print(f"Colors: {example['colors']}")
print(f"Moves: {example['moves']}")
print(f"Theory: {example['theory']}")
print(f"Target: {example['target']}")

print(f"\nShapes:")
print(f"Colors shape: {example['colors'].shape}")
print(f"Moves shape: {example['moves'].shape}")

Total training examples: 60600

First training example:
Colors: tensor([0, 0, 0, 0, 0, 1])
Moves: tensor([1927, 1927, 1927, 1927, 1927, 1730])
Theory: tensor([0, 0, 0, 0, 0, 1])
Target: 348

Shapes:
Colors shape: torch.Size([6])
Moves shape: torch.Size([6])


### Purge K-Fold Cross-Validation
Setup  Purge cross-validation splits for robust model evaluation and for avoiding data leaks. 

In [ ]:
# time based split to 5
n_splits = 5

#TimeSeriesSplit (time order)
tscv = TimeSeriesSplit(n_splits=n_splits)

# gives index range for each fold
splits = tscv.split(range(len(dataset)))

# train test split
fold_num = 1

for split in splits:
    train = split[0]
    test = split[1]


    fold_num += 1